# Immigration Data ETL
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* **Step 2: Explore and Assess the Data**
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

## Step 2 Explore and assess the data

### Overview
In this section we will explore the datasets chosen by **Flyby Salads** product management department using Apache Spark. We will idenfify and implement necessary transformations and finally export the data to S3 in parquet format.

Subsections of this Notebook:
* Preparations (imports, file locations, etc.)
* Reading files using Spark
* Data Exploration and description
    * Dataframe schema analysis
    * Data Quality analysis of the data
    * Data Type transformation steps
* Defining a new schema and storing it in JSON format
* Logical Data exploration examples
  (Analyzing table contents using PySpark to find more transformation actions)
    * Creating an airport dimension table
    * Linking airports to immigration data
    * 
* Exporting the resulting tables to Parquet format

**Note:**

    * We want to keep track of transformation steps
    * So if a required transformation is identified, it will be assigned a number like "TFA-x"
    * Where "TFA" stands for transformation action (plus a number)

### Preparations

#### Imports and Installs

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import logging
import sys
from datetime import datetime
from os.path import getsize
from nb_helpers import summarize_data, get_sas_definitions, read_sas_in_chunks, \
                        read_csv_print, print_stat, non_iso_date_change, change_nullables
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import pandas_udf

# Logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s %(levelname)s \t %(message)s ',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
log = logging.getLogger('log')

# Improve view
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


### Reading data files into Spark Dataframes
The next cell contains the file locations for each dataset, you may adjust it if required. Also note the configuration of the sample size which will improve performance while making some basic analysis.

In [2]:
# Source file locations
dem_file = 'us-cities-demographics.csv'
imm_file = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
air_file = 'airport-codes_csv.csv'

# Schema file locations
air_schema_file = 'air_schema.json'
dem_schema_file = 'dem_schema.json'
imm_schema_file = 'imm_schema.json'


# If working on a sample of the large immigration dataset, set your sample size here:
sample_size = 0.01

During my analysis I changed the schema of each dataset imported and based on the transformations done created a new schema definition. The definition is stored in a JSON file.

The data will be **imported again** using those schema **at the end** of the "Explore and Assess" section.

Now we setup the Spark Session and read all three files into a dataframe

In [21]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11") \
                    .enableHiveSupport() \
                    .getOrCreate()
start_time = datetime.now()

# Read data from sources
imm_df = spark.read \
                .format('com.github.saurfang.sas.spark') \
                .option("inferSchema","true") \
                .load(imm_file)
imm_rows = imm_df.count()
air_df = spark.read.csv(air_file, header=True)
air_rows = air_df.count()
dem_df = spark.read.csv(dem_file, header=True)
dem_rows = dem_df.count()

# List of our staging dataframes
df_list = {'imm_df': imm_df, 'air_df': air_df, 'dem_df': dem_df}

# Save a copy of initial total data, because we will work with a sample only in this notebook
imm_full = imm_df

# Output of results
elapse = datetime.now() - start_time
print('Reading ', imm_rows , ' lines from ', imm_file)
print('Reading ', dem_rows, ' lines from ', dem_file)
print('Reading ', air_rows, ' lines from ', air_file)
print('Operation runtime ', elapse)

Reading  3096313  lines from  ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
Reading  2891  lines from  us-cities-demographics.csv
Reading  55075  lines from  airport-codes_csv.csv
Operation runtime  0:00:27.576090


### Explore and Describe data
This section contains the following subsections:
* Spark schema analysis
* Spark dataframe descriptions analysis

Before we start this section we are naming each dataframe and reducing immigration data to a sample of the complete dataframe, keeping the full set in variable `imm_full` (the sample size being configured at the beginning of this notebook).

In [4]:
imm_df = imm_full.sample(sample_size)
imm_df.name = 'i94 immigration data'
dem_df.name = 'city demographics'
air_df.name = 'airport codes'

#### Display schemas created automatically by Spark

In [5]:
imm_df.printSchema()
air_df.printSchema()
dem_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

**Note** (TFA-1) Several columns in the i94 dataframe were infered as double while they should be of data type integer. Those types should be changed (see `cicid, i94yr,...`).

**Note** (TFA-2) Several columns in airport and demographic data were infered as string where they should be either double or integer (see airport data, `elevation_ft` or demographic data `median_age`

**Note** (TFA-3) Demographics data file was imported as a one-column file due to ';' being used as delimiter. We read the file again:

In [6]:
dem_df = spark.read.csv(dem_file, header=True, sep=';')
dem_df.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



**Note**: (TFA-4) For easier handling we strip columns of spaces, apply lower case and replace inner spaces and '-' with underscore '_'

In [7]:
def trim_col_headers(df):
    for col in df.columns:
        changed_col = col.strip().lower().replace(' ', '_').replace('-', '_')
        df = df.withColumnRenamed(col, changed_col)
    return df

dem_df = trim_col_headers(dem_df)

### Dataset descriptions

#### Statistical summary on data
The descriptive statistics were shown in Step 1 already, so we will skip this step here

#### Missing values per dataset
First we are measuring the amount of empty cells per column

In [8]:
# Get total amount of entries per column
imm_col_count = imm_full.describe().filter(F.col('summary') == 'count')
air_col_count = air_df.describe().filter(F.col('summary') == 'count')
dem_col_count = dem_df.describe().filter(F.col('summary') == 'count')

In [9]:
# Divide number of entries per column by total dataframe rows to get a percentage value
for df, numrows in zip([imm_col_count, air_col_count, dem_col_count], \
                       [imm_rows, air_rows, dem_rows]):
    for column in df.columns:
        if column == 'summary':
            next
        else:
            df = df.withColumn(column, F.col(column).cast('int'))
            df = df.withColumn(column, F.bround((F.col(column) / numrows * 100), scale=1))
    print('{}'.format(df.show()))

+-------+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+
|summary|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender|insnum|airline|admnum|fltno|visatype|
+-------+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+
|  count|100.0|100.0| 100.0| 100.0| 100.0|  100.0|  100.0|  100.0|   95.1|   95.4| 100.0|  100.0|100.0|   100.0|    39.2|  0.3|  100.0|   95.5|    0.0|   95.5|  100.0|  100.0|  86.6|   3.7|   97.3| 100.0| 99.4|   100.0|
+-------+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------

#### Unique values per column
We can count the number of unique values per dataframe or column using `dropDuplicates()` and `count()`

In [10]:
# Count **distinct** entries per column and print
for df, numrows in zip([imm_df, air_df, dem_df], [imm_rows, air_rows, dem_rows]):
    for column in df.columns:
        distinct_count = df.select(F.col(column)).distinct().count()
        print('No. of unique entries in table "{}": {}'.format(column, distinct_count))
    # print('{}'.format(df.show()))

No. of unique entries in table "cicid": 31085
No. of unique entries in table "i94yr": 1
No. of unique entries in table "i94mon": 1
No. of unique entries in table "i94cit": 182
No. of unique entries in table "i94res": 186
No. of unique entries in table "i94port": 148
No. of unique entries in table "arrdate": 30
No. of unique entries in table "i94mode": 5
No. of unique entries in table "i94addr": 93
No. of unique entries in table "depdate": 173
No. of unique entries in table "i94bir": 96
No. of unique entries in table "i94visa": 3
No. of unique entries in table "count": 1
No. of unique entries in table "dtadfile": 65
No. of unique entries in table "visapost": 247
No. of unique entries in table "occup": 21
No. of unique entries in table "entdepa": 13
No. of unique entries in table "entdepd": 11
No. of unique entries in table "entdepu": 2
No. of unique entries in table "matflag": 2
No. of unique entries in table "biryear": 96
No. of unique entries in table "dtaddto": 274
No. of unique entr

### Data Transformations
This section will contain the steps required to transform the datasets.

From previous notes we now carry out some transformation actions.
1. Double to Integer (TFA-1)
1. String to numbers (TFA-2)
1. SAS Date Type to date (TFA-5)
1. Non-Iso Date Type to date (TFA-6)

Note that TFA-3 and TFA-4 were already carried out in the previous chapter.

#### Integer Type Transformation (TFA-1)
The fields mentioned here are IntegerType but were infered as double, we are changing them to integer.

In addition the string column `entdepa` will also be changed.

In [22]:
int_columns = ['cicid','i94yr', 'i94mon', 'i94cit', 'i94res', \
              'arrdate', 'depdate', 'i94mode', 'i94bir', \
              'i94visa', 'count', 'biryear']

def change_type(df=None, fields=None, totype=None):
    if ((totype is not None) and (fields is not None)):
        for column in fields:
            cur_type = df.schema[column]
            df = df.withColumn(column, F.col(column).cast(totype))
            new_type = df.schema[column]
            print('Done, switched column {} from {} to {}' \
                  .format(column, cur_type.dataType, new_type.dataType))
        print(df.select(fields).show(5))
        return df
    else:
        return None

imm_df = change_type(imm_df, int_columns, 'int')
imm_df = change_type(imm_df, ['admnum'], 'bigint')

Done, switched column cicid from DoubleType to IntegerType
Done, switched column i94yr from DoubleType to IntegerType
Done, switched column i94mon from DoubleType to IntegerType
Done, switched column i94cit from DoubleType to IntegerType
Done, switched column i94res from DoubleType to IntegerType
Done, switched column arrdate from DoubleType to IntegerType
Done, switched column depdate from DoubleType to IntegerType
Done, switched column i94mode from DoubleType to IntegerType
Done, switched column i94bir from DoubleType to IntegerType
Done, switched column i94visa from DoubleType to IntegerType
Done, switched column count from DoubleType to IntegerType
Done, switched column biryear from DoubleType to IntegerType
+-----+-----+------+------+------+-------+-------+-------+------+-------+-----+-------+
|cicid|i94yr|i94mon|i94cit|i94res|arrdate|depdate|i94mode|i94bir|i94visa|count|biryear|
+-----+-----+------+------+------+-------+-------+-------+------+-------+-----+-------+
|    6| 2016| 

#### String to numbers (TFA-2)
In dataframes for demographic and airport data no float types were infered. Instead we see floating point numbers imported as strings.
Those will be changed here.

In [12]:
# Airport data:
float_columns = ['elevation_ft']
int_columns = None

air_df = change_type(air_df, float_columns, 'double')

Done, switched column elevation_ft from StringType to DoubleType
+------------+
|elevation_ft|
+------------+
|        11.0|
|      3435.0|
|       450.0|
|       820.0|
|       237.0|
+------------+
only showing top 5 rows

None


In [13]:
# Demographic Data
float_columns = ['median_age', 'average_household_size', 'count' ]
int_columns = ['male_population', 'female_population', \
               'total_population', 'number_of_veterans', 'foreign_born']

dem_df = change_type(dem_df, float_columns, 'double')
dem_df = change_type(dem_df, int_columns, 'int')

Done, switched column median_age from StringType to DoubleType
Done, switched column average_household_size from StringType to DoubleType
Done, switched column count from StringType to DoubleType
+----------+----------------------+-------+
|median_age|average_household_size|  count|
+----------+----------------------+-------+
|      33.8|                   2.6|25924.0|
|      41.0|                  2.39|58723.0|
|      38.5|                  2.58| 4759.0|
|      34.5|                  3.18|24437.0|
|      34.6|                  2.73|76402.0|
+----------+----------------------+-------+
only showing top 5 rows

None
Done, switched column male_population from StringType to IntegerType
Done, switched column female_population from StringType to IntegerType
Done, switched column total_population from StringType to IntegerType
Done, switched column number_of_veterans from StringType to IntegerType
Done, switched column foreign_born from StringType to IntegerType
+---------------+-------------

#### SAS Date Type Transformations (TFA-5)
The SAS formatted dates (e.v. "20573.0") should be changed to proper date types.
Note we are not putting this into the nb_helpers.py because it' s a more exemplary kind of transformation.

In [23]:
sas_date_columns = ['arrdate', 'depdate']
from pyspark.sql.functions import date_add

def sasdate_to_date(df=None, column_list=None):
    # SAS has its own epoch, which we add temporarily as a column
    df = df.withColumn('epoch_start', F.lit("01-01-1960 00:00:00"))
    df = df.withColumn('epoch_start', F.to_date(F.col('epoch_start'), "dd-M-yyyy"))
    # Check each column and convert double to int, then add this int to epoch_start
    for column in column_list:
        df = df.withColumn(column, F.col(column).cast('int'))
        stm = 'date_add(epoch_start, {})'.format(column)
        df = df.withColumn(column, F.expr(stm))
    return df

imm_df = sasdate_to_date(imm_df, sas_date_columns)
imm_df.select(sas_date_columns).show(5)

+----------+----------+
|   arrdate|   depdate|
+----------+----------+
|2016-04-29|      null|
|2016-04-07|      null|
|2016-04-01|2016-08-25|
|2016-04-01|2016-04-23|
|2016-04-01|2016-04-23|
+----------+----------+
only showing top 5 rows



#### Non-Iso Date Type Transformations (TFA-6)
Columns `dtadtto` and `dtadtofile` contain non-iso dateformats and will be converted into proper date types as well.

In [24]:
# if required, update the dictionary with a new column and a new pattern or change existing
imm_non_iso_dates = {'dtaddto': 'MMddyyyy', 'dtadfile': 'yyyyMMdd'}    
imm_df = non_iso_date_change(imm_df, imm_non_iso_dates)

print('Here a view on some examples: ')
print(imm_df.select('dtaddto').show(5), imm_df.select('dtadfile').show(5))

### Changing SAS Date Type to Spark Datetype
Done, switched column dtaddto from StringType to DateType
Done, switched column dtadfile from StringType to DateType
Here a view on some examples: 
+----------+
|   dtaddto|
+----------+
|2016-10-28|
|      null|
|2016-09-30|
|2016-09-30|
|2016-09-30|
+----------+
only showing top 5 rows

+----------+
|  dtadfile|
+----------+
|      null|
|2013-08-11|
|2016-04-01|
|2016-04-01|
|2016-04-01|
+----------+
only showing top 5 rows

None None


#### Missing values and nullable setting (TFA-7)

##### Changing nullable setting
Changing columns in the i94 dataset to `nullable = true` where applicable.

In [16]:
# Define fields which should not be empty
not_null_fields = ['cicid', 'admnum']
imm_df = imm_df.dropna(subset=not_null_fields)
imm_df = change_nullables(session=spark, df=imm_df, column_list=not_null_fields)

Done, Schema changed for  2  columns.


Demographic file did not have any missing values, so we set all fields to nullable = false

In [17]:
not_null_fields = dem_df.columns
dem_df = dem_df.na.drop(subset=not_null_fields)
dem_df = change_nullables(session=spark, df=dem_df, column_list=not_null_fields)

Done, Schema changed for  12  columns.


From airport data we want the items with iata code filled only

In [18]:
not_null_fields = ['iata_code']
air_df = air_df.na.drop(subset=not_null_fields)
air_df = change_nullables(session=spark, df=air_df, column_list=not_null_fields)

Done, Schema changed for  1  columns.


#### Exporting a PySpark schema to JSON
**(optional: re-importing data with updated schema from JSON)**
Now that we have altered the dataframe schemas we export those schemas into JSON files.

In [19]:
# Write to Json file using a function
def write_schema_json(df, schema_file):
    import json
    import sys
    with open(schema_file, "w") as f:
        json.dump(df.schema.jsonValue(), f)

# Let's call that for each dataframe
write_schema_json(imm_df, imm_schema_file)
write_schema_json(dem_df, dem_schema_file)
write_schema_json(air_df, air_schema_file)

**At this point we could read the JSON schema and apply it for importing data.**

*However during my work I got a ClassCastException after importing the data on ".show()" or any other actions.
This is why the following cell is turned into markdown and exempted from execution*

#Save some lines by using a function to import a schema from JSON

`def import_schema(schema_file):`   
    `import json
    schema_df = spark.sparkContext.wholeTextFiles(schema_file)
    schema_content = schema_df.collect()[0][1]
    schema_dict = json.loads(str(schema_content))
    new_schema = T.StructType.fromJson(schema_dict)
    return new_schema`

#Now apply the schema for each
`start_time = datetime.now()
imm_df = spark.read.format('com.github.saurfang.sas.spark').load(path=imm_file, schema=import_schema(imm_schema_file))
air_df = spark.read.csv(air_file, header=True, schema=import_schema(air_schema_file))
dem_df = spark.read.csv(dem_file, header=True, sep=';', schema=import_schema(dem_schema_file))
elapse = datetime.now() - start_time
print('Reading ', imm_df.count(), ' lines from ', imm_file)
print('Reading ', dem_df.count(), ' lines from ', dem_file)
print('Reading ', air_df.count(), ' lines from ', air_file)
print('Operation runtime ', elapse)`

#### Resulting schema definitions
In the sections above we have transformed specific values 

In [20]:
imm_df.printSchema()
air_df.printSchema()
dem_df.printSchema()

root
 |-- cicid: integer (nullable = false)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: date (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: date (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- dtadfile: date (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: long (nullable =

### Logical exploration using PySpark
In this chapter we will show some more explorative analysis that we help us understanding the data.

**While doing this we will carry out more transformations. Those will be designated again as "TFA-x" to keep track of them**

#### Example 01 - Cleaning airport data to get an airport dimension table (TFA-8)
The airport codes table has over 50k entries. We can reduce it by filtering for *type airport* and for matching with immigration data we also need the *iata_code*.

The resulting dataframe from the airport dataset will be kept as a transformed version of the table (trimmed of spaces in column `iata_code` and exported to Parquet later (dimension table "airport").

The code below creates five examples of travellers and their corresponding airport information.

In [21]:
# Read all available airport codes from the airport dataframe
airports_fields = ['iata_code', 'name', 'municipality', 'iso_region', \
                   'iso_country', 'coordinates']
dim_airports = air_df.filter(air_df.iata_code.isNotNull()).select(airports_fields)

# Split the coordinates column into latitude and longitude, remove spaces
dim_airports = dim_airports.withColumn('latitude', F.split(dim_airports['coordinates'], ',').getItem(0)).withColumn('longitude', F.split(dim_airports['coordinates'], ',').getItem(1))
dim_airports = dim_airports.withColumn('iata_code', F.trim(F.col('iata_code'))).withColumn('latitude', F.trim(F.col('latitude'))).withColumn('longitude', F.trim(F.col('longitude')))
dim_airports = dim_airports.drop('coordinates')
dim_airports.name = 'dim_airports'

# TEST if data can be matched
# Read some columns from immigration dataset including the airport codes, remove spaces
imm_airports = imm_df.select(['cicid', 'arrdate', 'i94port']).withColumnRenamed('i94port', 'airport').distinct()
imm_airports = imm_airports.withColumn('airport', F.trim(F.col('airport')))

# Join with immigration dataset
leftouter_set = imm_airports.join(dim_airports, dim_airports.iata_code == imm_airports.airport, 'leftouter')
matching = leftouter_set.select(['iata_code']).filter(F.col('iata_code').isNotNull()).count()
not_matching = leftouter_set.select(['iata_code']).filter(F.col('iata_code').isNull()).count()
print('Successfully joined lines: {} \t\t\tJoin failed on {} lines'.format(matching, (not_matching)))
print(leftouter_set.show(5))

Successfully joined lines: 20436 			Join failed on 10727 lines
+-------+----------+-------+---------+--------------------+------------+----------+-----------+------------------+-----------+
|  cicid|   arrdate|airport|iata_code|                name|municipality|iso_region|iso_country|          latitude|  longitude|
+-------+----------+-------+---------+--------------------+------------+----------+-----------+------------------+-----------+
|4041803|2016-04-22|    BGM|      BGM|Greater Binghamto...|  Binghamton|     US-NY|         US|      -75.97979736|42.20869827|
|  93624|2016-04-01|    FMY|      FMY|          Page Field|  Fort Myers|     US-FL|         US|-81.86329650879999|26.58659935|
| 818333|2016-04-05|    FMY|      FMY|          Page Field|  Fort Myers|     US-FL|         US|-81.86329650879999|26.58659935|
|2553357|2016-04-14|    FMY|      FMY|          Page Field|  Fort Myers|     US-FL|         US|-81.86329650879999|26.58659935|
|2740149|2016-04-15|    FMY|      FMY|          

This catches **only about 65% of the entries** in our fact table.

Now we could also try to match the city name in the SAS values list to the airport`municipality` column, but we consider this even worse.

Therefore we check the airport codes in the immigration table which are not matching to column `i94port` and **count missed joins per airport**:

In [22]:
# Get list of missing entries where airport code could not match
not_matching = leftouter_set.select(['airport', 'iata_code']).filter(F.col('iata_code').isNull())
# Count number of missing entries
not_matching.groupBy('airport').count().orderBy('count', ascending=False).show()

+-------+-----+
|airport|count|
+-------+-----+
|    NYC| 4917|
|    HHW| 1374|
|    CHI| 1286|
|    FTL|  985|
|    LVG|  906|
|    WAS|  771|
|    SAI|  224|
|    SAJ|   84|
|    NOL|   50|
|    YGF|   30|
|    XXX|   29|
|    X96|   26|
|    PHU|   20|
|    YHC|    7|
|    INP|    5|
|    PHR|    3|
|    MIL|    3|
|    JKM|    3|
|    X44|    1|
|    EPI|    1|
+-------+-----+
only showing top 20 rows



**This list sets a priority for cleaning up our airport dataframe or immigration data.**

*Example: if we find out why "NYC" is not matching an entry of our airport CSV list we would have an additional 4.888 joins with our fact table(!)*

One way to improve data quality on airports would be updating the airport code table with those missing codes and their corresponding city names. Another way would be to check if the airport code in the immigration data is correct at all.

*At least for our example "NYC" it seems that this is not a proper iata_code. New York's airports have individual codes like "LGA" for LaGuardia Airport.*

*This indicates that we should **correct the data in the immigration table** with the correct value. This requires however to **somehow derive the correct iata_code from other columns in the table**.

I will take this data quality action **out of scope**.

#### Example 02 - Create status code dimension table (TFA-9)


In [23]:
# Extract unique entries of status flags from immigration dataframe
status_columns = ['entdepa', 'entdepd', 'entdepu', 'matflag']

dim_status = imm_full.select(status_columns) \
                .dropna() \
                .dropDuplicates() \
                .withColumn("status_flag_id", \
                            F.monotonically_increasing_id()) 
dim_status.name = 'dim_status'
dim_status.show()

+-------+-------+-------+-------+--------------+
|entdepa|entdepd|entdepu|matflag|status_flag_id|
+-------+-------+-------+-------+--------------+
|      H|      O|      U|      M|  120259084288|
|      T|      O|      U|      M|  146028888064|
|      Z|      O|      U|      M|  214748364800|
|      T|      N|      U|      M|  523986010112|
|      G|      I|      U|      M|  644245094400|
|      G|      Q|      U|      M|  712964571136|
|      Z|      I|      U|      M|  876173328384|
|      G|      N|      U|      M|  919123001344|
|      O|      N|      U|      M| 1211180777472|
|      O|      I|      U|      M| 1228360646656|
|      Z|      K|      U|      M| 1254130450432|
|      G|      J|      U|      M| 1520418422784|
|      T|      K|      U|      M| 1623497637888|
|      Z|      R|      U|      M| 1632087572480|
|      G|      O|      U|      M| 1640677507072|
|      O|      O|      U|      M| 1675037245440|
+-------+-------+-------+-------+--------------+



Now we check if we can match those to our airports dataframe using the short code in column `i94port`:

#### Example 03 - Time Data dimension table (TFA-10)
We create a dimension table from the provided immigration data

In [29]:
# Define the relevant fields where to collect date entries for time dimension table
dim_time_columns = ['arrdate', 'depdate', 'dtaddto', 'dtadfile']

def create_dim_time(df=None, columns=None):
    """ This functions stacks given columns containing dates
        upon another.
        Then it derives from those dates the days, weeks, months and the year.
        
        Returns: Dataframe with time dimension table
        """
    
    # Reduce given df to the provided columns and remove duplicates
    dates_in_col = df.select(columns).dropDuplicates()

    # Create a dataframe containing content of the 1st data column
    dim_time = spark.createDataFrame(dates_in_col.select(dim_time_columns[0]).distinct().collect())

    # Iterate through the other columns and unionize them with the first one (by one :-)
    for next_col in dim_time_columns[1:]:
        temp_df = spark.createDataFrame(dates_in_col.select(next_col).collect())
        dim_time = dim_time.union(temp_df).distinct()
            
    # Calculate remaining colums from data in 1st column
    dim_time = dim_time.withColumnRenamed(dim_time_columns[0], 'datestamp')
    dim_time = dim_time \
                .withColumn('day_of_month', F.dayofmonth('datestamp')) \
                .withColumn('day_of_year', F.dayofyear('datestamp')) \
                .withColumn('week', F.weekofyear('datestamp')) \
                .withColumn('month', F.month('datestamp')) \
                .withColumn('year', F.year('datestamp')) \
                .dropDuplicates()
    return dim_time

#dim_time = create_dim_time(imm_df, dim_time_columns)
dim_time = create_dim_time(imm_df, dim_time_columns)
dim_time.name = 'dim_time'


print('Table {} has {} entries:'.format(dim_time.name, dim_time.count()))
print(dim_time.show(5))

Table dim_time has 837 entries:
+----------+------------+-----------+----+-----+----+
| datestamp|day_of_month|day_of_year|week|month|year|
+----------+------------+-----------+----+-----+----+
|2016-08-16|          16|        229|  33|    8|2016|
|2016-09-16|          16|        260|  37|    9|2016|
|2018-04-17|          17|        107|  16|    4|2018|
|2017-02-27|          27|         58|   9|    2|2017|
|2016-06-04|           4|        156|  22|    6|2016|
+----------+------------+-----------+----+-----+----+
only showing top 5 rows

None


#### Example 04 - Demographics dimension table (TFA-11)

We aggregate the demographics data per state (column `state_code`) and match it to the immigration data.


In [25]:
# Select ALL columns for this dimension table into dimension table
demographics_columns = dem_df.columns
dim_demographics = dem_df.select(demographics_columns).dropDuplicates()

# Define grouping function per column 
cols_to_mean = 'average_household_size'
cols_to_sum = ['state_code', 'male_population', 'female_population', \
               'total_population', 'number_of_veterans', 'foreign_born']

# Calculate weighted average per state (arithmetic average not applicable)
def weighted_average(df=None, mean_column=None, weight_column=None, agg_level=None):
    """ Calculate weighted average for a dataframe.
        You need to provide the column which stores the average values
        and the column storing the weights.
        You can specify an aggregation level which is a column by which the
        data will be grouped (resulting in weighted averages for each group)
        
        Returns: Dataframe with weighted average column
        """
    # Reduce to required columns:
    df = df.select([mean_column, weight_column, agg_level]) \
            .withColumnRenamed(mean_column, 'mean_col') \
            .withColumnRenamed(weight_column, 'weight_col') \
            .withColumnRenamed(agg_level, 'agg_col')
    
    # Sum up weight column grouped by aggregation level
    sum_of_weights = df.select(['agg_col', 'weight_col']).groupBy('agg_col').sum()
    sum_of_weights = sum_of_weights.withColumnRenamed('sum(weight_col)', 'sum_weights')
    
    # Calculate product of individual means multiplied by weights
    mean_times_weight = df.withColumn('mean_weight_product', (F.col('mean_col') * (F.col('weight_col'))))
    # Sum up all mean values per agg level
    sum_of_means = mean_times_weight.select(['agg_col', 'mean_weight_product']).groupBy('agg_col').sum()
    sum_of_means = sum_of_means.withColumnRenamed('sum(mean_weight_product)', 'sum_of_meanpr')
    
    # Divide aggregated means by total weights per agg level
    weighted_mean = sum_of_means \
                    .join(sum_of_weights, 'agg_col') \
                    .withColumn('weighted_avg', (F.col('sum_of_meanpr') / F.col('sum_weights')))
    # Send result back
    return_df = spark.createDataFrame(weighted_mean.select(['agg_col', 'weighted_avg']).collect())
    return return_df


weighted_mean_df = weighted_average(dim_demographics, cols_to_mean, 'total_population', 'state_code')
summed_columns_df = dim_demographics.select(cols_to_sum).groupBy('state_code').sum()

dim_demographics = summed_columns_df.join(weighted_mean_df, weighted_mean_df.agg_col == summed_columns_df.state_code)
dim_demographics = dim_demographics.drop('agg_col') \
                    .withColumnRenamed('sum(male_population)', 'male_population') \
                    .withColumnRenamed('sum(female_population)', 'female_population') \
                    .withColumnRenamed('sum(total_population)', 'total_population') \
                    .withColumnRenamed('sum(number_of_veterans)', 'number_of_veterans') \
                    .withColumnRenamed('sum(foreign_born)', 'foreign_born') \
                    .withColumnRenamed('weighted_avg', 'average_household_size')

dim_demographics.name = 'dim_demographics'
dim_demographics.show(3)                                            

+----------+---------------+-----------------+----------------+------------------+------------+----------------------+
|state_code|male_population|female_population|total_population|number_of_veterans|foreign_born|average_household_size|
+----------+---------------+-----------------+----------------+------------------+------------+----------------------+
|        AZ|       11137275|         11360435|        22497710|           1322525|     3411565|     2.771264313123426|
|        SC|        1265291|          1321685|         2586976|            163334|      134019|     2.471309525098029|
|        LA|        3134990|          3367985|         6502975|            348855|      417095|    2.4628252838124087|
+----------+---------------+-----------------+----------------+------------------+------------+----------------------+
only showing top 3 rows



#### Example 05 - Immigration Fact Table (TFA-12

We now prepare the fact table for export. During this the fact table is updated with a status code for dimension table `dim_status`

In [26]:
# List of columns which are used to join with dim_status
join_status_condition = [dim_status.entdepa == imm_df.entdepa, dim_status.entdepu == imm_df.entdepu, \
                         dim_status.entdepd == imm_df.entdepd, dim_status.matflag == imm_df.matflag]
join_columns = ['entdepa', 'entdepu', 'entdepd', 'matflag']

fact_df = imm_df.join(dim_status, join_columns, 'leftouter')

immigration_facts = fact_df.withColumn("cicid", F.col("cicid").cast("integer")) \
                    .withColumn('status', F.col('status_flag_id')) \
                    .withColumnRenamed('admnum', 'adm_number') \
                    .withColumnRenamed('arrdate','arrival_dt') \
                    .withColumnRenamed('depdate','departure_dt') \
                    .withColumn("cit_country", F.col("i94cit").cast("integer")) \
                    .withColumn("res_country", F.col("i94res").cast("integer")) \
                    .withColumnRenamed("i94port", "airport") \
                    .withColumn("transport_mode", F.col("i94mode").cast("integer")) \
                    .withColumnRenamed("i94addr", "state") \
                    .withColumn("age", F.col("i94bir").cast("integer")) \
                    .withColumn("visatype", F.col("i94visa").cast("integer")) \
                    .drop('i94yr', 'i94mon', 'i94cit', 'i94res', 'arrdate', \
                          'depdate', 'count', 'admnum', 'i94mode','i94bir' \
                          'i94bir', 'dtadfile', 'epoch_start', 'status'
                          'dtadfile', 'visapost', 'dtaddto', 'insnum', 'i94visa')

imm_df_ommitted_columns = ['i94yr', 'i94mon', 'dtadfile', 'epoch_start', 'status'
                          'dtadfile', 'visapost', 'dtaddto', 'insnum', 'i94visa']

immigration_fact_fields = ['cicid', 'status', 'adm_number', 'transport_mode', 'airport', \
                           'state', 'arrival_dt', 'departure_dt', 'airline', \
                           'fltno', 'visatype', 'age', 'gender', 'res_country', \
                           'cit_country', 'occup']


immigration_facts = immigration_facts.select(immigration_fact_fields)
immigration_facts.name = 'immigration_facts'
immigration_facts.show(5)

+-------+------+-----------+--------------+-------+-----+----------+------------+-------+-----+--------+---+------+-----------+-----------+-----+
|  cicid|status| adm_number|transport_mode|airport|state|arrival_dt|departure_dt|airline|fltno|visatype|age|gender|res_country|cit_country|occup|
+-------+------+-----------+--------------+-------+-----+----------+------------+-------+-----+--------+---+------+-----------+-----------+-----+
|2645795|  null|93511289830|             1|    LOS|   CA|2016-04-14|        null|     Y4|00912|       2| 35|     M|        582|        582| null|
|5682339|  null|59541506933|             1|    SPM|   MN|2016-04-30|        null|     DL|00163|       1| 60|     M|        131|        123| null|
|     52|  null|92469910630|             1|    NYC|   NY|2016-04-01|  2016-04-14|     AB|07450|       2| 54|     M|        112|        101| null|
|     79|  null|55433479933|             1|    BOS|   MA|2016-04-01|  2016-04-05|     LH|00422|       1| 51|     M|        1

### Exporting the data to Parquet
We export the collected data in Parquet format after we have processed the cleaning steps above.
Note that in this notebook we are only writing a sample of the immigration data. Dataframe `imm_full` contains all 3M rows we will write in production.

In [27]:
def write_to_parquet(df=None, location='', name='', s3loc=''):
    """ Writes provided dataframe content to parquet files
        Use "location" variable to specify folder/sub-folders
        and "name" for the parquet filename.
        Optionally: provide an S3 bucket, the function will then
        write to the S3 location
        
        Returns: Nothing
    """
    # Put together an output location link
    location = s3loc +location + name
    try:
        print('Trying to write {} to location: {}'.format(name, location))
        df.write.parquet(location, mode='overwrite', compression='gzip')
        print('Done writing')
    except Exception as e:
        print(datetime.now(), ': Writing output files failed: ', e)

folder = 'temp/'
s3loc = ''

for table in table_list [immigration_facts, dim_demographics, dim_airports, dim_time, dim_status]:
    write_to_parquet(df=table, location=folder, name=table.name)

Trying to write immigration_facts to location: temp/immigration_facts
Done writing
Trying to write dim_demographics to location: temp/dim_demographics
Done writing
Trying to write dim_airports to location: temp/dim_airports
Done writing
Trying to write dim_time to location: temp/dim_time
Done writing
Trying to write dim_status to location: temp/dim_status
Done writing
